# Imports

In [3]:
import numpy as np
import pandas as pd
import os
import re
import time
from difflib import SequenceMatcher

from gensim.models import Word2Vec
from tqdm import tqdm

import nltk
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer

import sys

Using TensorFlow backend.


# Importação dos dados

In [8]:
df = pd.read_csv('base_chatbot.csv', encoding = 'utf-8').drop("Unnamed: 0", axis = 1)

In [9]:
df

,frases
0,rj
1,pena maria jose percebi estado voce falando na...
2,voce deseje falar armazem paraiba regiao <SITE>
3,deseja continuar conversa clique continuar
4,sim
...,...
49440,preciso informe cpf pessoa realizou compra
49441,compra amilton status pedido
49442,alguma informacao sobre pedido
49443,ajudar


In [10]:
processed_array = df['frases']

print('Separando as sequencias de treino')

train_sentences = list(processed_array.apply(str.split).values)

print(len(train_sentences))

Separando as sequencias de treino
49445


In [11]:
train_sentences

[['rj'],
 ['pena',
  'maria',
  'jose',
  'percebi',
  'estado',
  'voce',
  'falando',
  'nao',
  'possui',
  'lojas',
  'armazem',
  'paraiba',
  'claudino',
  'atuamos',
  'apenas',
  'estados',
  'pb',
  'pe',
  'ce',
  'ba'],
 ['voce', 'deseje', 'falar', 'armazem', 'paraiba', 'regiao', '<SITE>'],
 ['deseja', 'continuar', 'conversa', 'clique', 'continuar'],
 ['sim'],
 ['maria', 'jose', 'vamos', 'conversar', 'sobre'],
 ['favor', 'escolha', 'opcoes', 'abaixo'],
 ['pedido'],
 ['ajudar',
  'preciso',
  'numero',
  'pedido',
  'compra',
  '<PEDIDO>',
  'voce',
  'pode',
  'encontra',
  'mail',
  'enviamos',
  'compra',
  'realizada'],
 ['<PEDIDO>'],
 ['preciso', 'informe', 'cpf', 'pessoa', 'realizou', 'compra'],
 ['<NUMERO>'],
 ['nao', 'consegui', 'encontrar', 'compras', 'cpf', 'informa', 'novamente'],
 ['<PEDIDO>'],
 ['nao', 'entendi', 'voce', 'quis', 'dizer', 'vamos', 'tentar', 'novamente'],
 ['favor', 'escolha', 'opcoes', 'abaixo'],
 ['pedido'],
 ['ajudar',
  'preciso',
  'numero',
 

# Treinamento do modelo

In [18]:
start_time = time.time()

model = Word2Vec(sentences = train_sentences,
                 sg=1, 
                 size=100,  
                 workers=4,
                 min_count=1)

print(f'Time taken : {(time.time() - start_time) / 60:.2f} mins')

Time taken : 0.03 mins


In [19]:
len(model.wv.vocab.keys())

4191

In [14]:
word_set = set()

df['frases'].str.lower().str.split().apply(word_set.update)

print(len(word_set))

untrained = []

for w in word_set:
    if w not in list(model.wv.vocab.keys()):
        untrained.append(w)

len(untrained)

4079


2657

In [23]:
model.wv.vocab.keys()

dict_keys(['rj', 'pena', 'maria', 'jose', 'percebi', 'estado', 'voce', 'falando', 'nao', 'possui', 'lojas', 'armazem', 'paraiba', 'claudino', 'atuamos', 'apenas', 'estados', 'pb', 'pe', 'ce', 'ba', 'deseje', 'falar', 'regiao', '<SITE>', 'deseja', 'continuar', 'conversa', 'clique', 'sim', 'vamos', 'conversar', 'sobre', 'favor', 'escolha', 'opcoes', 'abaixo', 'pedido', 'ajudar', 'preciso', 'numero', 'compra', '<PEDIDO>', 'pode', 'encontra', 'mail', 'enviamos', 'realizada', 'informe', 'cpf', 'pessoa', 'realizou', '<NUMERO>', 'consegui', 'encontrar', 'compras', 'informa', 'novamente', 'entendi', 'quis', 'dizer', 'tentar', '<CPF>', 'insira', 'dados', 'possamos', 'entrar', 'contato', 'jamis', 'status', 'alguma', 'informacao', 'ainda', 'aprendendo', 'algumas', 'duvidas', 'consigo', 'aqui', 'fala', 'duvida', 'prazo', 'entrega', 'confirmacao', 'pagamento', 'disponivel', 'retirada', 'loja', 'comprou', 'retirar', 'produto', 'horas', 'uteis', 'apos', 'receber', 'consultar', 'finalizacao', 'ate', '

# Testes

In [22]:
model.wv.most_similar('pedido', topn = 20)

[('<PEDIDO>', 0.63839191198349),
 ('enviamos', 0.6176563501358032),
 ('selma', 0.5902417898178101),
 ('cavalcante', 0.5880042314529419),
 ('realizada', 0.5855375528335571),
 ('casos', 0.57552170753479),
 ('karoline', 0.5754918456077576),
 ('cancelados', 0.5649179220199585),
 ('identifiquei', 0.5632619857788086),
 ('encontra', 0.5618027448654175),
 ('enviaremos', 0.5604972839355469),
 ('todos', 0.5566429495811462),
 ('francicleide', 0.5541908740997314),
 ('atualizacao', 0.5519514679908752),
 ('thaynan', 0.5497175455093384),
 ('holanda', 0.5460906624794006),
 ('esposo', 0.5445817112922668),
 ('ajuda', 0.5429922938346863),
 ('kleberson', 0.5417298078536987),
 ('giorgio', 0.5412965416908264)]

In [24]:
model.wv.most_similar('entregue', topn = 20)

[('sera', 0.9272394180297852),
 ('<span', 0.8698452115058899),
 ('color', 0.8459981679916382),
 ('border', 0.8129509091377258),
 ('partir', 0.8127745985984802),
 ('<h', 0.8106753826141357),
 ('style', 0.8075528144836426),
 ('solid', 0.8073062896728516),
 ('amanha', 0.8027127981185913),
 ('finalizacao', 0.7953567504882812),
 ('estara', 0.7937850952148438),
 ('resposta', 0.7835416197776794),
 ('você', 0.7827049493789673),
 ('dei', 0.7794210910797119),
 ("green;'><b>06/05/2020</b></h>", 0.7780252695083618),
 ('pegar', 0.777721643447876),
 ('entrada', 0.7766943573951721),
 ("green;'><b>29/01/2020</b></h>", 0.7765957117080688),
 ('deveria', 0.7758206129074097),
 ('identidade', 0.775371253490448)]

In [29]:
model.wv.most_similar('curriculo', topn = 20)

[('responsavel', 0.924424409866333),
 ('realizado', 0.9240573644638062),
 ('serto', 0.9167047739028931),
 ('clico', 0.9141855835914612),
 ('rose', 0.9096769094467163),
 ('he', 0.9059473276138306),
 ('dodia', 0.9058418273925781),
 ('busca', 0.9046359658241272),
 ('atrasado', 0.8996473550796509),
 ('vc', 0.8992165327072144),
 ('segunda', 0.8964459896087646),
 ('comer', 0.8935270309448242),
 ('solucao', 0.8926879167556763),
 ('mandei', 0.8912091851234436),
 ('entrrega', 0.8910342454910278),
 ('espelho', 0.8902729153633118),
 ('pedidp', 0.8901995420455933),
 ('enviou', 0.8884093165397644),
 ('demolicao', 0.8876223564147949),
 ('mandar', 0.8873152732849121)]